Use this notebook to pull in datasets and apply pre-processing.  Most grammar datasets unfortunately require preprocessing before being usable in training. (example - jfleg has 4 targets per input, so we have to rematch as 1:1 pairings) 

In [1]:
import csv
from datasets import load_metric, load_dataset
from pathlib import Path

In [2]:
list_replacements = [
  (" .", "."), 
  (" ,", ","),
  (" '", "'"),
  (" ?", "?"),
  (" !", "!"),
  (" :", "!"),
  (" ;", "!"),
  (" n't", "n't"),
  (" v", "n't"),
  ("2 0 0 6", "2006"),
  ("5 5", "55"),
  ("4 0 0", "400"),
  ("1 7-5 0", "1750"),
  ("2 0 %", "20%"),
  ("5 0", "50"),
  ("1 2", "12"),
  ("1 0", "10"),
  ('" ballast water', '"ballast water')
  ]

In [3]:
def correct_spacing(item):
    """ we iterate through the list of all replacements per each item in dataset"""
    for fix in list_replacements:
        item = item.replace(fix[0], fix[1])
    return item



In [4]:
def generate_csv(csv_path, dataset):
    """ apply spacing corrections and save out matched pairs to csv file as dataset"""
    with open(csv_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["input", "target"])
        for case in dataset:
     	    # Adding the t5 task indication prefix to input 
            input_text = "grammar: " + case["sentence"]
            input_text = correct_spacing(input_text)

            for correction in case["corrections"]:
              correction = correct_spacing(correction)
              # a few of the cases contain blank strings. 
              if input_text and correction:
                writer.writerow([input_text, correction])

Jfleg is an odd setup - validation will be used as 'train', test will be 'validation'

In [5]:
train_dataset = load_dataset("jfleg", split='validation[:]') 
eval_dataset = load_dataset("jfleg", split='test[:]')


Using custom data configuration default
Reusing dataset jfleg (/home/ubuntu/.cache/huggingface/datasets/jfleg/default/1.0.0/ed4ab2367351fe31949f48849ae6732b164f0d5ea6bb5d4357ff4293ac89511b)
Using custom data configuration default
Reusing dataset jfleg (/home/ubuntu/.cache/huggingface/datasets/jfleg/default/1.0.0/ed4ab2367351fe31949f48849ae6732b164f0d5ea6bb5d4357ff4293ac89511b)


In [6]:
print(train_dataset)
print(eval_dataset)


Dataset({
    features: ['sentence', 'corrections'],
    num_rows: 755
})
Dataset({
    features: ['sentence', 'corrections'],
    num_rows: 748
})


In [11]:
print(train_dataset['sentence'][22])
print(train_dataset['corrections'][22])

Students can focus on only a few subjects they are intwerested in and they will become an experts in those areas . 
['Students can focus on only a few subjects they are interested in and they will become experts in those areas . ', 'Students can focus on only a few subjects they are interested in and they will become experts in those areas . ', 'Students can focus on only a few subjects they are interested in and they will become an expert in those areas . ', 'Students can focus on only a few subjects they are interested in and they will become an expert in those areas . ']


In [7]:
clean22 = correct_spacing(train_dataset['sentence'][22])
clean22

'Students can focus on only a few subjects they are intwerested in and they will become an experts in those areas. '

In [8]:
Path.cwd()

PosixPath('/home/ubuntu/t5_11/datasets_grammar')

In [16]:
jfleg_dir = Path.cwd()/'jfleg_dataset'  # if you only use 'jfleg', hf will try and use that and complain
jfleg_dir.mkdir(parents=True,exist_ok=True)
c4_dir = Path.cwd()/'c4_dataset'
c4_dir.mkdir(parents=True,exist_ok=True)

Process Jfleg data  

In [17]:
j_train_file = jfleg_dir/'jtrain.csv'
j_eval_file = jfleg_dir/'jeval.csv'

In [18]:
generate_csv(j_train_file, train_dataset)

In [20]:
generate_csv(j_eval_file, eval_dataset)

Process C4_200M (!) - we'll pull 10K to start

In [9]:
c4_dataset = load_dataset("liweili/c4_200m", streaming = True)

Using custom data configuration default


In [10]:
iterator = iter(c4_dataset['train'])

In [11]:
def c4_generate_csv(csv_path, iterator, num_examples):
    with open(csv_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["input", "target"])
        for i in range(0,num_examples):
          data = next(iterator)
          input_text = "grammar: " + data["input"]
          input_text = correct_spacing(input_text)
          correction = correct_spacing(data["output"])
          if input_text and correction:
            writer.writerow([input_text, correction])

In [12]:
c4_dir = Path.cwd()/'c4_dataset'
c4_dir.mkdir(parents=True,exist_ok=True)

In [31]:
c4_filename = c4_dir/'c4train_10M.csv'

In [32]:
c4_generate_csv(c4_filename, iterator, num_examples=10000000)

Create a single training file by combining jtrain and c4train

In [33]:
merge_list = [j_train_file, c4_filename, ]

In [34]:
import pandas as pd

In [35]:
combo_csv = pd.concat([pd.read_csv(fn) for fn in merge_list])


In [36]:
merged_name = "gtrain_10M.csv"

In [37]:
combo_csv.to_csv(merged_name, index=False, encoding = 'utf-8-sig', )

In [4]:
import torch
from torch.distributed.fsdp import *

In [5]:
r=100
print(f"test {r}")

test 100
